# Proceso de ETL para el dataset "MOVIES"

In [1]:
%run lib.ipynb

## 1.- Extracción, desde archivos de datos originales

In [2]:
data = pd.read_csv("datasets/movies_dataset.csv")
original_shape = data.shape

/tmp/ipykernel_116791/2289977068.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("datasets/movies_dataset.csv")


## 2.- Transformación

### Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [3]:
#crea nuevo dataframe para calcular las recomendaciones
data_recomendaciones = pd.DataFrame(data, columns=['overview', 'tagline', 'title', 'spoken_languages', 'release_date']).copy()

#Realizaré la función de recomendación sobre el campo overview, por lo que necesito que esté poblado 
#para todas las películas
print('Suma de registros nulos en la columna overview: ', data_recomendaciones["overview"].isnull().sum())
print('Suma de registros nulos en la columna tagline: ', data_recomendaciones["tagline"].isnull().sum())

#Reemplazo los valores nulos de la columna overview por su correspondiente en tagline
data_recomendaciones["overview"] = data_recomendaciones["overview"].fillna(data_recomendaciones["tagline"])
print('Suma de registros nulos en la columna overview: ', data_recomendaciones["overview"].isnull().sum())

#Reemplazo los valores nulos de la columna overview por su correspondiente en title
data_recomendaciones["overview"] = data_recomendaciones["overview"].fillna(data_recomendaciones["title"])
print('Suma de registros nulos en la columna overview: ', data_recomendaciones["overview"].isnull().sum())

#Eliminar columnas tagline y title
data_recomendaciones = data_recomendaciones.drop(['tagline'], axis=1)

data_recomendaciones.reset_index(drop=True, inplace=True)
data_recomendaciones["overview"] = data_recomendaciones["overview"].str.lower()
data_recomendaciones["title"] = data_recomendaciones["title"].str.lower()

#reducir el tamaño del contenido de la columna overview para disminuir el tamaño de la matriz de recomendaciones
#data_recomendaciones["overview"] = data_recomendaciones["overview"].apply(lambda x: textwrap.shorten(x, 150, placeholder=''))

#eliminar peliculas que no esten habladas en ingles
print(data_recomendaciones['spoken_languages'].str.contains("English").sum())
data_recomendaciones = data_recomendaciones[data_recomendaciones['spoken_languages'].str.contains("English") == True]

#eliminar peliculas con release_date < '01-01-2015'
data_recomendaciones = data_recomendaciones[data_recomendaciones['release_date'] >= '2015-01-01']

data_recomendaciones.reset_index(drop=True, inplace=True)
data_recomendaciones = data_recomendaciones.drop(['spoken_languages', 'release_date'], axis=1)
print(data_recomendaciones.shape)

write_parquet(data_recomendaciones, 'datasets/data_recomendaciones_dataset.parquet')



Suma de registros nulos en la columna overview:  954
Suma de registros nulos en la columna tagline:  25054
Suma de registros nulos en la columna overview:  946
Suma de registros nulos en la columna overview:  0
28745
(2654, 2)


### Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [4]:
print(f'{data["revenue"].isnull().sum()} valores nulos en la columna "revenue" antes de llenarlos con 0') 
data["revenue"] = data["revenue"].fillna(0)
print(f'{data["revenue"].isnull().sum()} valores nulos en la columna "revenue" despues de llenarlos con 0')

print(f'{data["budget"].isnull().sum()} valores nulos en la columna "budget" antes de llenarlos con 0')
data["budget"] = data["budget"].fillna(0)
print(f'{data["budget"].isnull().sum()} valores nulos en la columna "budget" despues de llenarlos con 0')

6 valores nulos en la columna "revenue" antes de llenarlos con 0
0 valores nulos en la columna "revenue" despues de llenarlos con 0
0 valores nulos en la columna "budget" antes de llenarlos con 0
0 valores nulos en la columna "budget" despues de llenarlos con 0


### Los valores nulos del campo release date deben eliminarse.

In [5]:
print(f'{data["release_date"].isnull().sum()} valores nulos en la columna "release_date" antes de llenarlos con 0')
data["release_date"].info()
print('\n')
data = data.dropna(subset="release_date")
print(f'{data["release_date"].isnull().sum()} valores nulos en la columna "release_date" después de llenarlos con 0')
data["release_date"].info()

data.reset_index(drop=True, inplace=True)

87 valores nulos en la columna "release_date" antes de llenarlos con 0
<class 'pandas.core.series.Series'>
RangeIndex: 45466 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype 
--------------  ----- 
45379 non-null  object
dtypes: object(1)
memory usage: 355.3+ KB


0 valores nulos en la columna "release_date" después de llenarlos con 0
<class 'pandas.core.series.Series'>
Index: 45379 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype 
--------------  ----- 
45379 non-null  object
dtypes: object(1)
memory usage: 709.0+ KB


### De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.  

In [6]:
# Asegurar fechas en formato AAAA-mm-dd
release_date_series = pd.Series(data["release_date"])

s=pd.to_datetime(release_date_series, format='%Y-%m-%d', errors='coerce').isna()

print(data['release_date'].info())
print(f'\nLos registros con problemas en el formato de fecha son: {s.loc[s == True].index.shape[0]}')

data.iloc[s.loc[s == True].index]

<class 'pandas.core.series.Series'>
RangeIndex: 45379 entries, 0 to 45378
Series name: release_date
Non-Null Count  Dtype 
--------------  ----- 
45379 non-null  object
dtypes: object(1)
memory usage: 354.7+ KB
None

Los registros con problemas en el formato de fecha son: 3


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19714,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29472,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35543,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Los registros que no cumplen el formato de fecha son eliminados
data.drop(index = s.loc[s == True].index, inplace = True)
data.reset_index(drop=True, inplace=True)
print(data['release_date'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 45376 entries, 0 to 45375
Series name: release_date
Non-Null Count  Dtype 
--------------  ----- 
45376 non-null  object
dtypes: object(1)
memory usage: 354.6+ KB
None


In [8]:
#Agregar culumna release_year
data['release_year'] = data['release_date'].str.split('-').str[0].astype(int)
data['release_year'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45376 entries, 0 to 45375
Series name: release_year
Non-Null Count  Dtype
--------------  -----
45376 non-null  int64
dtypes: int64(1)
memory usage: 354.6 KB


### Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [9]:
data['return'] = np.where(((data['revenue'] == 0) | (data['budget'].astype(float) == 0)), 0, data['revenue'].astype(float)/data['budget'].astype(float))
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45376 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45376 non-null  object 
 3   genres                 45376 non-null  object 
 4   homepage               7766 non-null   object 
 5   id                     45376 non-null  object 
 6   imdb_id                45362 non-null  object 
 7   original_language      45365 non-null  object 
 8   original_title         45376 non-null  object 
 9   overview               44435 non-null  object 
 10  popularity             45376 non-null  object 
 11  poster_path            45037 non-null  object 
 12  production_companies   45376 non-null  object 
 13  production_countries   45376 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,return
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,12.451801
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,4.043035
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,0.000000
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,5.090760
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,0.000000
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,3.123947
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,0.000000
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,0.000000
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995,1.838576
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995,6.072311


### Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [10]:
data.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)
print(f'El formato original del datafreme al inicio ', original_shape, ' y al final ', data.shape)

El formato original del datafreme al inicio  (45466, 24)  y al final  (45376, 20)


## 3.- Carga, genera archivo resultado en formato parquet

In [11]:
# Errores al generar archivo parquet

#ArrowInvalid: (\"Could not convert '0.837228' with type str: tried to convert to double\", 'Conversion failed for column popularity with type object')"
#Asiganaré 0 a la columna popularity cuando el valor no sea numérico.
data['popularity'] = pd.to_numeric(data['popularity'], errors='coerce')
data['popularity'] = data['popularity'].fillna(0)
data['popularity'] = data['popularity'].astype(float)

write_parquet(data, 'datasets/movies_dataset.parquet')